2nd version, working on the 106 compounds that are unmapped after Roberts help!

In [4]:
import os
import pandas as pd
import re
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
# import the excel 
check_in = "/home/jackmcgoldrick/openTECR/data/2025_openTECR_hackathon_check-in.xlsx"

In [4]:
# convert to df
comp_matching = pd.read_excel(check_in, sheet_name="compound matching")

In [5]:
comp_matching

,compound_name,kegg,appears in part 1,appears in part 2,appears in part 3,appears in part 4,appears in part 5,appears in part 6,appears in part 7
0,benzyl alcohol,C00556,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NAD,C00003,1.0,1.0,1.0,1.0,NaN,1.0,NaN
2,benzaldehyde,C00261,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NADH,C00004,1.0,1.0,NaN,1.0,NaN,1.0,NaN
4,1-butanol,C06142,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
687,thiamine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688,thiopyrophosphate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689,UDP-D-fucose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
690,UDP-D-quinovose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# filter those which have no kegg - move to sep df
unmapped_comps = comp_matching[comp_matching['kegg'].isnull()]

In [7]:
unmapped_comps

,compound_name,kegg,appears in part 1,appears in part 2,appears in part 3,appears in part 4,appears in part 5,appears in part 6,appears in part 7
585,(3R)-3-hydroxyhexanoyl-CoA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
586,(S)-(+)-2-octanol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587,(S)-2-heptanol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
588,1-glycerol 3-phosphate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
589,15-oxo-prostaglandin E2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
687,thiamine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688,thiopyrophosphate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689,UDP-D-fucose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
690,UDP-D-quinovose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# has the correct number of mets i.e 106

# now want to extract the compound_name into list
compounds = unmapped_comps['compound_name']

In [9]:
compounds

585         (3R)-3-hydroxyhexanoyl-CoA
586                  (S)-(+)-2-octanol
587                     (S)-2-heptanol
588             1-glycerol 3-phosphate
589            15-oxo-prostaglandin E2
                    ...               
687                           thiamine
688                  thiopyrophosphate
689                       UDP-D-fucose
690                    UDP-D-quinovose
691    uridine 3':5'-(cyclic)phosphate
Name: compound_name, Length: 107, dtype: object

In [10]:
def reconXMapper_name(name):
    """
    Query which returns best matches related to an input molecule's name.

    This query implements a strategy to:
    1. Look for exact matches (case-insensitive) and assign priority = 1.
    2. If no exact matches exist, suggest partial matches (priority = 2).

    Parameters:
        name (str): The molecule name to search for.

    Returns:
        List of dictionaries with 'mainName', 'coalescedValue', and 'priority'.
    """
    # Define the SPARQL endpoint
    sparql_endpoint = "http://sbg:7200/repositories/ReconXKG"
    sparql = SPARQLWrapper(sparql_endpoint)
    sparql.setReturnFormat(JSON)

    # SPARQL query
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX mnx: <https://rdf.metanetx.org/schema/>
    PREFIX reconx: <https://reconx.vital-it.ch/kg/>

    SELECT DISTINCT 
           ?mainName
           (COALESCE(?xrefIDs, ?exactName, ?partialName, "N/A") AS ?coalescedValue)
           ?priority
    WHERE {{
      ##############################################################################
      # Subquery 1: Retrieve the main chem info (?chemID, ?mainName, ?xrefIDs)
      ##############################################################################
      {{
        SELECT ?chemID ?mainName ?xrefIDs
        WHERE {{
          ?chemID a mnx:CHEM ;
                  rdfs:comment ?mainName .

          BIND(?chemID AS ?tempChemID)

          # Possibly multiple cross-reference IDs
          {{
            ?tempChemID mnx:chemXref ?xrefIDs .
          }}
          UNION
          # Fallback to binding the chem ID itself
          {{
            BIND(?tempChemID AS ?xrefIDs)
          }}
        }}
      }}

      ##############################################################################
      # Subquery 2: Exact match => priority=1
      ##############################################################################
      {{
        SELECT ?chemID (1 AS ?priority) ?exactName
        WHERE {{
          ?chemID a mnx:CHEM ;
                  rdfs:comment ?exactName .

          FILTER(REGEX(?exactName, "^{name}$", "i"))
        }}
      }}
      UNION
      ##############################################################################
      # Subquery 3: Partial match => priority=2
      ##############################################################################
      {{
        SELECT ?chemID (2 AS ?priority) ?partialName
        WHERE {{
          ?chemID a mnx:CHEM ;
                  mnx:chemXref/rdfs:comment ?partialName .

          FILTER(REGEX(?partialName, "\\b{name}\\b", "i"))

          # Exclude partial matches if an exact match exists globally
          FILTER NOT EXISTS {{
            ?anyChem a mnx:CHEM ;
                     rdfs:comment ?anyExactName .

            FILTER(REGEX(?anyExactName, "^{name}$", "i"))
          }}
        }}
      }}
    }}
    ORDER BY ?priority
    """

    # Set the query string and execute
    sparql.setQuery(query)
    try:
        # Execute the query
        results = sparql.query().convert()
        bindings = results.get("results", {}).get("bindings", [])
        
        # Simplify the output
        simplified_results = []
        for result in bindings:
            simplified_results.append({
                "mainName": result.get("mainName", {}).get("value", "N/A"),
                "coalescedValue": result.get("coalescedValue", {}).get("value", "N/A"),
                "priority": int(result.get("priority", {}).get("value", "2"))  # Default to priority 2
            })
        
        return simplified_results
    except Exception as e:
        raise RuntimeError(f"SPARQL query failed: {e}")

## Test 1: Run Query on 1 compound

In [21]:
# Search for a specific compound name
compound_name = "L-mannose"  # Replace this with the compound name you want to search for
matching_compounds = compounds[compounds == compound_name]

# Assuming matching_compounds is a pandas Series
test_compound = matching_compounds.iloc[0]  # Extract the first value

# Pass the extracted compound name to your function
result = reconXMapper_name(test_compound)

# Print the result
print(result)

[{'mainName': 'L-mannose', 'coalescedValue': 'https://identifiers.org/CHEBI:37676', 'priority': 1}, {'mainName': 'L-mannose', 'coalescedValue': 'http://purl.obolibrary.org/obo/CHEBI_37676', 'priority': 1}, {'mainName': 'L-mannose', 'coalescedValue': 'https://rdf.metanetx.org/chem/MNXM59481', 'priority': 1}]


In [9]:
def run_query_and_save_to_csv(series, output_path):
    """
    Run the SPARQL query for each compound in the pandas Series and save the results to a CSV file.

    Parameters:
        series (pd.Series): Series containing the compound names.
        output_path (str): Full file path to save the output CSV.

    Returns:
        None
    """
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    results = []

    for compound in series:
        try:
            # Run the query for the current compound
            query_result = reconXMapper_name(compound)
            
            # Simplify the output into a readable string
            readable_result = "; ".join(
                [
                    f"{res['mainName']} ({res['coalescedValue']}, Priority {res['priority']})"
                    for res in query_result
                ]
            )
        except Exception as e:
            readable_result = f"Error: {e}"

        # Append the compound and result to the list
        results.append({"compound": compound, "query_result": readable_result})

    # Convert to DataFrame and save to the specified file path
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")


In [24]:
# define output_dir to store csv
output_dir = "/home/jackmcgoldrick/openTECR/results/compounds_mapped_ver1.csv"

run_query_and_save_to_csv(compounds, output_dir)

Results saved to /home/jackmcgoldrick/openTECR/results/compounds_mapped_ver1.csv


## Attempting to Map Remaining Rxns (470)

In [2]:
# import new csv with extra mets

new_data = "/home/jackmcgoldrick/openTECR/data/freeText_toMap.xlsx"

In [5]:
# read the correct sheet
new_mets = pd.read_excel(new_data, sheet_name='compound matching v2')

In [6]:
new_mets

,compound_name,kegg,appears in part 1,appears in part 2,appears in part 3,appears in part 4,appears in part 5,appears in part 6,appears in part 7
0,benzyl alcohol,C00556,1.0,NaN,NaN,NaN,NaN,NaN,1.0
1,NAD,C00003,1.0,1.0,1.0,1.0,NaN,1.0,NaN
2,benzaldehyde,C00261,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NADH,C00004,1.0,1.0,NaN,1.0,NaN,1.0,NaN
4,1-butanol,C06142,1.0,NaN,1.0,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
1187,3-phenylpropionitrile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1188,2-phenylpropanoic acid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1189,(deoxyribonucleotide)_{n},NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1190,(deoxyribonucleotide)_{m},NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [7]:
# return entries where "kegg" is empty
compounds_unmapped = new_mets[new_mets['kegg'].isnull()]

In [8]:
compounds_unmapped

,compound_name,kegg,appears in part 1,appears in part 2,appears in part 3,appears in part 4,appears in part 5,appears in part 6,appears in part 7
649,desamino NAD,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
650,desamino NADH,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
658,D-glycero-D-glucoheptitol,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
661,2-oxo-myo-inositol,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
663,2-oxo-4-hydroxybutanoate,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1187,3-phenylpropionitrile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1188,2-phenylpropanoic acid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1189,(deoxyribonucleotide)_{n},NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1190,(deoxyribonucleotide)_{m},NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [11]:
compounds_2 = compounds_unmapped['compound_name']

In [12]:
compounds_2

649                    desamino NAD
650                   desamino NADH
658       D-glycero-D-glucoheptitol
661              2-oxo-myo-inositol
663        2-oxo-4-hydroxybutanoate
                   ...             
1187          3-phenylpropionitrile
1188         2-phenylpropanoic acid
1189      (deoxyribonucleotide)_{n}
1190      (deoxyribonucleotide)_{m}
1191    (deoxyribonucleotide)_{m+n}
Name: compound_name, Length: 471, dtype: object

In [13]:
# 470 entries unmapped and to run in ReconXKG
def query_sparql(endpoint_url, name_of_interest):
    """
    Query a SPARQL endpoint for chemical information based on the name of interest.

    :param endpoint_url: str, the SPARQL endpoint URL.
    :param name_of_interest: str, the exact name to search for (case-insensitive).
    :return: list of tuples, each containing the chemical ID and exact name.
    """
    # Define the SPARQL query with the name of interest
    sparql_query = f"""
    PREFIX mnx: <https://rdf.metanetx.org/schema/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?chemID ?exactName
    WHERE {{
      ?chemID a mnx:CHEM ;
              rdfs:comment ?exactName .
      FILTER(REGEX(?exactName, "^{name_of_interest}$", "i"))
    }}
    """

    # Initialize the SPARQL wrapper
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    try:
        # Execute the query and process the results
        results = sparql.query().convert()
        output = []
        for result in results["results"]["bindings"]:
            chem_id = result["chemID"]["value"]
            exact_name = result["exactName"]["value"]
            output.append((chem_id, exact_name))
        return output
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [19]:
import os
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

def query_sparql(endpoint_url, name_of_interest):
    """
    Query a SPARQL endpoint for chemical information based on the name of interest.

    :param endpoint_url: str, the SPARQL endpoint URL.
    :param name_of_interest: str, the exact name to search for (case-insensitive).
    :return: list of dictionaries, each containing chemical ID and exact name.
    """
    # Escape the name_of_interest to avoid SPARQL injection or errors
    escaped_name = name_of_interest.replace('"', '\\"')

    sparql_query = f"""
    PREFIX mnx: <https://rdf.metanetx.org/schema/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?chemID ?exactName
    WHERE {{
      ?chemID a mnx:CHEM ;
              rdfs:comment ?exactName .
      FILTER(REGEX(?exactName, "^{escaped_name}$", "i"))
    }}
    """

    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.query().convert()
        output = []
        for result in results["results"]["bindings"]:
            output.append({
                "chemID": result["chemID"]["value"],
                "exactName": result["exactName"]["value"]
            })
        return output
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Query that caused the error:")
        print(sparql_query)
        return []

def run_query_and_save(series, endpoint_url, output_path):
    """
    Run the SPARQL query for each compound in the pandas Series and save the results to a CSV file.

    Parameters:
        series (pd.Series): Series containing the compound names.
        endpoint_url (str): The SPARQL endpoint URL.
        output_path (str): Full file path to save the output CSV.

    Returns:
        None
    """
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    results = []

    for compound in series:
        try:
            # Run the query for the current compound
            query_result = query_sparql(endpoint_url, compound)

            # Simplify the output into a readable string
            readable_result = "; ".join(
                [
                    f"{res['exactName']} ({res['chemID']})"
                    for res in query_result
                ]
            ) if query_result else "No results found"
        except Exception as e:
            readable_result = f"Error: {e}"

        # Append the compound and result to the list
        results.append({"compound": compound, "query_result": readable_result})

    # Convert to DataFrame and save to the specified file path
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

In [21]:

# Example usage
if __name__ == "__main__":
    # Replace with the actual SPARQL endpoint URL
    endpoint_url = "http://sbg:7200/repositories/ReconXKG"

   
    # Output file path
    output_csv_path = "/home/jackmcgoldrick/openTECR/results/chemical_query_results.csv"

    # Run the query and save results
    run_query_and_save(compounds_2, endpoint_url, output_csv_path)


An error occurred: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'MALFORMED QUERY: Lexical error at line 8, column 37.  Encountered: \'97\' (97), after prefix "\\"^5{\\\\"'
Query that caused the error:

    PREFIX mnx: <https://rdf.metanetx.org/schema/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?chemID ?exactName
    WHERE {
      ?chemID a mnx:CHEM ;
              rdfs:comment ?exactName .
      FILTER(REGEX(?exactName, "^5{\alpha}-pregnane-3{\alpha},17{\alpha},21-triol-20-one$", "i"))
    }
    
An error occurred: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'MALFORMED QUERY: Lexical error at line 8, column 37.  Encountered: \'97\' (97), after prefix "\\"^5{\\\\"'
Query that caused the error:

    PREFIX mnx: <https://rdf.metanetx.org/schema/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?chemID